In [1]:
# first import the module
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

In [2]:
#then have news and behavior data ready
# read news data
news = pd.read_csv(
    "C:/Users/ryan0/OneDrive/Obsidian_Lib/11_GWU/13_24FA/CSCI_6365_A_ML/20240908_AML_HW1/Dataset/archive/MINDsmall_train/news.tsv", 
    sep="\t",
    names=["itemId", "category", "subcategory", "title", "abstract", "url", "title_entities", "abstract_entities"]
)
news.head(2)

,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."


In [3]:
# read behaviours data
raw_behaviour = pd.read_csv(
    "C:/Users/ryan0/OneDrive/Obsidian_Lib/11_GWU/13_24FA/CSCI_6365_A_ML/20240908_AML_HW1/Dataset/archive/MINDsmall_train/behaviors.tsv", 
    sep="\t",
    names=["impressionId","userId","timestamp","click_history","impressions"])
raw_behaviour.head(2)

,impressionId,userId,timestamp,click_history,impressions
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...


In [4]:
# Create mappings between indices and item IDs
ind2item = {idx + 1: itemid for idx, itemid in enumerate(news['itemId'].values)}
item2ind = {itemid: idx for idx, itemid in ind2item.items()}

# Map subcategories to unique IDs
unique_subcategories = news['subcategory'].unique()
subcategory_to_id = {subcategory: idx + 1 for idx, subcategory in enumerate(unique_subcategories)}
news['subcategoryId'] = news['subcategory'].map(subcategory_to_id)

# Create mappings between subcategory IDs and news IDs
subcatid_to_news = news.groupby('subcategoryId')['itemId'].apply(list).to_dict()
news_to_subcatid = news.set_index('itemId')['subcategoryId'].to_dict()


---
then process behaviors


In [5]:
# Function to process impressions
def process_impression(s):
    list_of_strings = s.split(" ")
    itemid_rel_tuple = [l.split("-") for l in list_of_strings]
    noclicks = []
    clicks = []
    for entry in itemid_rel_tuple:
        if entry[1] == '0':
            noclicks.append(entry[0])
        elif entry[1] == '1':
            clicks.append(entry[0])
    return noclicks, clicks

# Apply the function to the 'impressions' column
raw_behaviour[['noclicks', 'clicks']] = raw_behaviour['impressions'].apply(lambda x: pd.Series(process_impression(x)))

# Process 'click_history' column
raw_behaviour['click_history'] = raw_behaviour['click_history'].apply(lambda x: x.split(" ") if pd.notna(x) else [])

# Display the dataset
print(f"The dataset consists of {len(raw_behaviour)} interactions after processing.")


The dataset consists of 156965 interactions after processing.


In [6]:
# Function to aggregate user preferences
def aggregate_user_preferences(group):
    click_history = sum(group['click_history'], [])
    clicks = sum(group['clicks'], [])
    noclicks = sum(group['noclicks'], [])
    return pd.Series({
        'history': click_history,
        'clicks': clicks,
        'noclicks': noclicks
    })

# Group by 'userId' and aggregate
preference = raw_behaviour.groupby('userId').apply(aggregate_user_preferences).reset_index()


preference.sample(5)#randomly sample 5 rows



C:\Users\ryan0\AppData\Local\Temp\ipykernel_14832\131946685.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  preference = raw_behaviour.groupby('userId').apply(aggregate_user_preferences).reset_index()


,userId,history,clicks,noclicks
34675,U68928,"[N33375, N43675, N40526, N26176, N39598, N4681...","[N26706, N39758, N61022, N49279, N49685]","[N21882, N14520, N44123, N53245, N64513, N5761..."
36494,U7196,"[N34309, N51238, N26975, N40716, N17050, N3124...","[N41220, N41222]","[N3930, N38783, N39758, N11363, N46666, N64513..."
49564,U93920,"[N60300, N52551, N18360, N41218, N60300, N5255...","[N18312, N49180, N63970, N3123]","[N6066, N25295, N36660, N31448, N26025, N14029..."
18844,U41926,"[N54103, N26176, N306, N43142, N28257, N54103,...","[N53585, N41165, N16920]","[N55689, N14029, N56214, N47061, N35047, N5041..."
38094,U74591,"[N17933, N11071, N21623, N60206, N25114, N5170...","[N36681, N21707, N1034, N56214, N50675, N47652]","[N41224, N50060, N49685, N26952, N29128, N5520..."


In [7]:
# Initialize a user-news interaction matrix
user_ids = preference['userId'].unique()  
news_ids = news['itemId'].unique()  

# Create mappings 
user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
news_id_to_index = {news_id: idx for idx, news_id in enumerate(news_ids)}

# Initialize user-news matrix
interaction_matrix = np.zeros((len(user_ids), len(news_ids)))


for row in preference.itertuples():
    user_idx = user_id_to_index[row.userId]
    for news_id in row.history:  # only history
        if news_id in news_id_to_index:
            news_idx = news_id_to_index[news_id]
            interaction_matrix[user_idx, news_idx] = 1


In [8]:

# Convert to sparse
interaction_sparse = csr_matrix(interaction_matrix)

k = 50  # Number of latent features
U, sigma, Vt = svds(interaction_sparse, k=k)
sigma = np.diag(sigma)

# convert to float32 to reduce memory usage
U = U.astype(np.float32)
sigma = sigma.astype(np.float32)
Vt = Vt.astype(np.float32)

# predicted rating matrix
predicted_matrix = np.dot(np.dot(U, sigma), Vt)


In [9]:
predicted_df = pd.DataFrame(predicted_matrix, index=user_ids, columns=news_ids)

def recommend_news_svd(user_id, predicted_df, preference, top_n=100):
    user_idx = user_id_to_index[user_id]
    user_scores = predicted_df.iloc[user_idx].sort_values(ascending=False)
    
    user_history = set(preference[preference['userId'] == user_id]['history'].values[0])
    
    recommended_news = [news_id for news_id in user_scores.index if news_id not in user_history]
    
    return recommended_news[:top_n]

# test single user
user_id = 'U2'
recommended_news = recommend_news_svd(user_id, predicted_df, preference, top_n=100)

print(f"Top recommended news for user {user_id}: {recommended_news}")


Top recommended news for user U2: ['N42458', 'N12096', 'N58090', 'N477', 'N40716', 'N62058', 'N2203', 'N55556', 'N22561', 'N3566', 'N19347', 'N23653', 'N56253', 'N30974', 'N32004', 'N21317', 'N18030', 'N23958', 'N26364', 'N62006', 'N59496', 'N40692', 'N54698', 'N46513', 'N20121', 'N51238', 'N60050', 'N23614', 'N33976', 'N44399', 'N64273', 'N36019', 'N40509', 'N63429', 'N39462', 'N53895', 'N60985', 'N4020', 'N6218', 'N54536', 'N19638', 'N18355', 'N22816', 'N33998', 'N619', 'N36699', 'N56742', 'N4607', 'N11101', 'N32482', 'N57582', 'N11071', 'N60699', 'N3259', 'N55805', 'N57801', 'N60844', 'N14724', 'N14340', 'N46023', 'N32483', 'N31801', 'N35009', 'N56447', 'N5227', 'N50710', 'N33247', 'N23554', 'N16874', 'N60206', 'N50032', 'N49229', 'N61319', 'N8448', 'N39041', 'N13636', 'N459', 'N44361', 'N33617', 'N61864', 'N38046', 'N4866', 'N55743', 'N11894', 'N57967', 'N62471', 'N18870', 'N61210', 'N32089', 'N38105', 'N61249', 'N5183', 'N1445', 'N31057', 'N20039', 'N47671', 'N24148', 'N2155', 'N6

In [11]:
def evaluate_recommendations(selected_user_ids, preference, predicted_df, top_n=100):
    recall_scores, precision_scores = [], []

    for user_id in selected_user_ids:
        recommended_news = recommend_news_svd(user_id, predicted_df, preference, top_n=top_n)
        if not recommended_news:
            continue
        
        actual_clicked_news = set(preference[preference['userId'] == user_id]['clicks'].values[0])
        if not actual_clicked_news:
            continue
        
        num_relevant = len(set(recommended_news) & actual_clicked_news)
        recall = num_relevant / len(actual_clicked_news) if actual_clicked_news else 0
        precision = num_relevant / len(recommended_news) if recommended_news else 0
        
        recall_scores.append(recall)
        precision_scores.append(precision)

    average_recall = sum(recall_scores) / len(recall_scores) if recall_scores else 0
    average_precision = sum(precision_scores) / len(precision_scores) if precision_scores else 0
    
    return average_recall, average_precision

selected_user_ids = random.sample(list(preference['userId']), 100)

average_recall, average_precision = evaluate_recommendations(selected_user_ids, preference, predicted_df, top_n=10)

print(f"Average Recall: {average_recall:.4f}")
print(f"Average Precision: {average_precision:.4f}")


Average Recall: 0.0033
Average Precision: 0.0010
